In [ ]:
%matplotlib inline

import os, sys, datetime, pickle, warnings
warnings.filterwarnings('ignore')

import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

import scipy.stats as ss
import statsmodels.api as sa
import scikit_posthocs as sp
import sklearn

import seaborn as sns

In [ ]:
module_path = os.getcwd()
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
### import custom functions
import settings as settings
import summary_helper as my_help

## Global variables and parameters

In [ ]:
""" ============== Golbal variables and parameters =============="""
### import path setting
paths = settings.paths

### settings about target and heel mapping.
matching_info = settings.matching_info

### settings about colormaps of graphs
color_code =  matching_info.color_code
color_code[0] = 'gray'
palette = {
    'R3':color_code[3],
    'R4':color_code[4],
    'R3/R3':color_code[3],
    'R4/R4':color_code[4]
}

### settings about figure outputs
is_save = False
fig_format = 'svg'
fig_res = 300

## Load Data

In [ ]:
"""data outputs from image processing pipeline"""
# Dictionaries with relative lengths and angles info for targets and heels of each bundle in each sample
# structure: {(time_id, sample_no):rel_points_sample}
#     - rel_points_sample: dictionary. {bundle_no:rel_points_bundle}
#           - rel_points_bundle: dictionary.
#                  - keys = 'T0c', 'T0l', 'T0h', 'T2c', 'T2l', 'T2h', 'T3c', 'T3l', 'T3h', 'T4c', 'T4l', 'T4h', 'T5c', 'T5l', 'T5h', 'T7c', 'T7l', 'T7h', 'center', 'R3', 'R4', 'length_one_um', 'phi_range_1', 'phi_range_2'
rel_pos_ctrl = pickle.load(open(os.path.join(paths.data_prefix, "rel_pos_ctrl.pickle"), "rb"))
rel_pos_fz = pickle.load(open(os.path.join(paths.data_prefix, "rel_pos_fz.pickle"), "rb"))
rel_pos_nic = pickle.load(open(os.path.join(paths.data_prefix, "rel_pos_nic.pickle"), "rb"))

# Dictionaries with intensity matrix and parameters of representative bundles to generate sub-figures.
# structure: {(figure_id, time_id, bundle_type ('ctrl' vs. 'pert')): bundle_data_dic}
# - bundle_data_dic: dictionary. keys = 'figure_ID', 'time_ID', 'bundle_type', 'category_ID', 'parameter', 'relative_positions', 'intensity_matrix', 'analysis_params_general'
#     - 'figure_ID', 'time_ID', 'category_ID', 'bundle_type': general information about this particular bundle.
#     - 'relative_positions': dictionary. relative positions of targets and heels of this particular bundle.
#     - 'intensity_matrix': numpy array. GFP and RFP density map of this particular bundle.
#     - 'analysis_params_general': class of settings.GeneralParams. parameters used when doing image quantification
output_data_ctrl_eg = pickle.load(open(os.path.join(paths.data_prefix, "output_data_ctrl_eg.pickle"), "rb"))
output_data_fz_eg = pickle.load(open(os.path.join(paths.data_prefix, "output_data_fz_eg.pickle"), "rb"))
output_data_nic_eg = pickle.load(open(os.path.join(paths.data_prefix, "output_data_nic_eg.pickle"), "rb"))


In [ ]:
""" annotation files """
# csv files contains manual annotations of images.
### controls
filepath = os.path.join(paths.data_prefix, 'annots_df_ctrl_sum.xlsx')
annots_df_ctrl = pd.read_excel(filepath)
annots_df_ctrl.fillna(0, inplace = True)

### Fzs
filepath = os.path.join(paths.data_prefix, 'annots_df_fz_sum.xlsx')
annots_df_fz = pd.read_excel(filepath)
annots_df_fz.fillna(0, inplace = True)

### Nics
filepath = os.path.join(paths.data_prefix, 'annots_df_nic_sum.xlsx')
annots_df_nic = pd.read_excel(filepath)
annots_df_nic.fillna(0, inplace = True)

In [ ]:
"""summary files"""
# csv files contains annotated growth cone length and angle. 
sum_df_ctrl = pd.read_excel(os.path.join(paths.data_prefix, 'summary_ctrl.xlsx'))
sum_df_ctrl = sum_df_ctrl.loc[~sum_df_ctrl['category_id'].isna(),:]

sum_df_fz = pd.read_excel(os.path.join(paths.data_prefix, 'summary_fz.xlsx'))
sum_df_fz = sum_df_fz.loc[~sum_df_fz['category_id'].isna(),:]

sum_df_nic = pd.read_excel(os.path.join(paths.data_prefix, 'summary_nic.xlsx'))
sum_df_nic = sum_df_nic.loc[~sum_df_nic['category_id'].isna(),:]

## Data Processing

In [ ]:
"""Process annotation files"""
# align coordinate data to center and T4, as well as add information from image processing
# pipeline.
print("=== process annotation files ===")
print("Control:", end = " ")
annots_df_ctrl_final = my_help.process_annots_df(annots_df_ctrl, rel_pos_ctrl)

print("Fz:", end = " ")
annots_df_fz_final = my_help.process_annots_df(annots_df_fz, rel_pos_fz)

print("Nic:", end = " ")
annots_df_nic_final = my_help.process_annots_df(annots_df_nic, rel_pos_nic)

In [ ]:
"""process summary files"""
## Combines image annotation information and growth cone angle and length annotation information.
## controls
print("=== process summary files ===")
print("Control:", end = " ")
sum_df_ctrl[['time_id', 'sample_no', 'bundle_no']] = sum_df_ctrl[['time_id', 'sample_no', 'bundle_no']].astype('int64')
sum_df_ctrl_final = my_help.process_sum_df(sum_df_ctrl, annots_df_ctrl_final, rel_pos_ctrl, False)
sum_df_ctrl_final.sort_values(by =['time_id', 'sample_no', 'bundle_no'], inplace = True)

# ### Fzs
print("\nFz:", end = " ")
sum_df_fz[['time_id', 'sample_no', 'bundle_no']] = sum_df_fz[['time_id', 'sample_no', 'bundle_no']].astype('int64')
sum_df_fz_final = my_help.process_sum_df(sum_df_fz, annots_df_fz_final, rel_pos_fz, False)
sum_df_fz_final.sort_values(by =['time_id', 'sample_no', 'bundle_no'], inplace = True)

# ### Nics
print("\nNic:", end = " ")
sum_df_nic[['time_id', 'sample_no', 'bundle_no']] = sum_df_nic[['time_id', 'sample_no', 'bundle_no']].astype('int64')
sum_df_nic_final = my_help.process_sum_df(sum_df_nic, annots_df_nic_final, rel_pos_nic, False)
sum_df_nic_final.sort_values(by =['time_id', 'sample_no', 'bundle_no'], inplace = True)

In [ ]:
sum_df_ctrl_final.columns

In [ ]:
"""regression analysis on repulsion model"""
print("=== get regression bundles ===")

sum_criteria = (sum_df_ctrl_final['symmetry']<=0.5) & (sum_df_ctrl_final['time_id']<=26)
sum_df_ctrl_ml_temp = sum_df_ctrl_final.loc[sum_criteria,:]
annots_criteria = (annots_df_ctrl_final['symmetry']<=0.5) & (annots_df_ctrl_final['time_id']<=26)
annots_df_ctrl_ml = annots_df_ctrl_final.loc[annots_criteria,:]

sum_df_ctrl_ml = my_help.process_sum_df(sum_df_ctrl_ml_temp, annots_df_ctrl_ml, rel_pos_ctrl, True)
sum_df_ctrl_ml.sort_values(by =['time_id', 'sample_no', 'bundle_no'], inplace = True)

print("\n=== regression analysis ===")
df_data = my_help.mutual_repulsion_regression(sum_df_ctrl_ml, annots_df_ctrl_ml)


## Main Figures

### Figure 2
Establishment of standardized coordinates for comparison of growth cone extension.

In [ ]:
### representative GFP density map in Figure 2.
my_help.generate_density_plot_figure(output_data_ctrl_eg[('Fig2', 22, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP'], gfp_cmap = [0.2, [0, 0.1, 0.2]],
                                     fig_name = 'Figure2', fig_format = fig_format, fig_res = fig_res)

### Figure 3
Wild-type R3s and R4s exhibit asymmetric speeds but symmetric directions of extension.

#### Figure 3A: Relative length of R3 and R4s from wild-type flies over time.

In [ ]:
# ====== Figure ====== #
### set parameters
df_current = sum_df_ctrl_final
x_cat = 'time_id'
y_cat = 'length_gc'
hue = 'type_plot'
df_plot = df_current[[x_cat, y_cat, hue]].dropna(how='any')
ylim = [0, 3]
ylabel = 'Relative Length (a.u.)'
xlabel = 'Time (hrs APF)'

### calculate T3/T4 range
times = len(df_current.groupby(x_cat).groups)
x = np.arange(times+2)-1
y = np.zeros((6, times+2))

y[0,1:-1] = df_current.groupby(x_cat).mean()['T4l_fromheel']
y[1,1:-1] = df_current.groupby(x_cat).mean()['T4h_fromheel']
y[2,1:-1] = df_current.groupby(x_cat).mean()['T4c_fromheel']
y[3,1:-1] = df_current.groupby(x_cat).mean()['T3l_fromheel']
y[4,1:-1] = df_current.groupby(x_cat).mean()['T3h_fromheel']
y[5,1:-1] = df_current.groupby(x_cat).mean()['T3c_fromheel']
y[:,0] = y[:,1]
y[:,-1] = y[:,-2]

### figure set-up
SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 28
plt.rc('font', size=SMALL_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)   # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE) # fontsize of the figure title

sns.set_style("white")

### plotting
fig, ax = plt.subplots(1,1, figsize = (12,8))
if('length' in y_cat):
    ax.plot(x, y[2,:], '--', color = color_code[4], label='_nolegend_')
    ax.plot(x, y[5,:], '--', color = color_code[3], label='_nolegend_')
    ax.fill_between(x, y[1,:], y[0,:], color=color_code[4], alpha=0.2, label='_nolegend_')
    ax.fill_between(x, y[3,:], y[4,:], color=color_code[3], alpha=0.2, label='_nolegend_')

sns.boxplot(x=x_cat, y=y_cat, data=df_plot, hue = hue, dodge = True, palette = palette, ax = ax)
sns.swarmplot(x=x_cat, y=y_cat, data=df_plot, color = '.25', hue = hue, dodge = True, ax = ax)
# g.legend.remove()
ax.set_ylim(ylim)
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
ax.tick_params(axis = 'y', labelleft = [0,1,2,3,4])

ax.legend(title = 'R type', loc='center left', bbox_to_anchor=(1.01, 0.9))

if(is_save):
    fig_name = f'Figure3B.{fig_format}'
    fig_save_path = os.path.join(paths.output_prefix, fig_name)
    plt.savefig(fig_save_path, dpi=fig_res, bbox_inches='tight', format = fig_format)

plt.show()

In [ ]:
# ====== p values of relative length ====== #

In [ ]:
pairs_list = [[22, 24], [24, 26], [26, 28], [28, 30], [30, 32], [32, 36]]

In [ ]:
r_type = "R3"
df_r = df_plot.groupby(hue).get_group(r_type)

data = [df_r.loc[ids, y_cat].values for ids in df_r.groupby(x_cat).groups.values()]
H, p = ss.kruskal(*data)
stats = sp.posthoc_mannwhitney(df_r, val_col=y_cat, group_col=x_cat, p_adjust = 'holm')

print("=== significance of R3 relative length ===")
for pair in pairs_list:
    stat = stats.loc[pair[0], pair[1]]
    print(f"{pair[0]} vs. {pair[1]}: {stat: .5f}", end = ", ")
    if(stat >= 0.05):
        print("ns")
    elif((stat < 0.05) & (stat >= 0.01)):
        print("*")
    elif((stat < 0.01) & (stat >= 0.001)):
        print("**")
    else:
        print("***")

In [ ]:
r_type = "R4"
df_r = df_plot.groupby(hue).get_group(r_type)

data = [df_r.loc[ids, y_cat].values for ids in df_r.groupby(x_cat).groups.values()]
H, p = ss.kruskal(*data)
stats = sp.posthoc_mannwhitney(df_r, val_col=y_cat, group_col=x_cat, p_adjust = 'holm')

print("=== significance of R4 relative length ===")
for pair in pairs_list:
    stat = stats.loc[pair[0], pair[1]]
    print(f"{pair[0]} vs. {pair[1]}: {stat: .5f}", end = ", ")
    if(stat >= 0.05):
        print("ns")
    elif((stat < 0.05) & (stat >= 0.01)):
        print("*")
    elif((stat < 0.01) & (stat >= 0.001)):
        print("**")
    else:
        print("***")

#### Figure 3B: Relative angle of R3 and R4s from wild-type flies over time.

In [ ]:
# ====== Figure ====== #
### set parameters
x_cat = 'time_id'
y_cat = 'angle_gc_plot'
hue = "type_plot"
xlim = 8

colors1 = plt.cm.Reds(np.linspace(0, 1, 15))
colors1 = colors1[5:,:]

colors2 = plt.cm.YlOrBr(np.linspace(0, 1, 15))
colors2 = colors2[3:,:]

phi_unit = my_help.get_angle_unit_theory('phi_unit')

### calculate values
r3_control = sum_df_ctrl_final.groupby(hue).get_group('R3')
r4_control = sum_df_ctrl_final.groupby(hue).get_group('R4')

theta1 = r3_control.groupby(x_cat).mean()[y_cat].to_numpy()
width1 = r3_control.groupby(x_cat).std()[y_cat].to_numpy()

theta2 = r4_control.groupby(x_cat).mean()[y_cat].to_numpy()
width2 = r4_control.groupby(x_cat).std()[y_cat].to_numpy()

radii1 = np.arange(len(theta1))+1
radii2 = np.arange(len(theta2))+1

theta1 = np.flip(theta1)
radii1 = np.flip(radii1)
width1 = np.flip(width1)
theta2 = np.flip(theta2)
radii2 = np.flip(radii2)
width2 = np.flip(width2)

dTiCs = {}
target_grid_polar = my_help.get_target_grid_polar_summary(return_type = 'theory', dTiCs = dTiCs)

### figure set-up
SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 28
plt.rc('font', size=SMALL_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)   # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE) # fontsize of the figure title


### plotting
fig = plt.figure(figsize=(12,8))
ax = fig.add_axes([0.1, 0.1, 0.75, 0.79], polar=True)

# plot reference line
ax.plot([0,0], [0,xlim], '--', color = 'Gray', linewidth = 1.5)
ax.plot([0,target_grid_polar[2,0]], [0,xlim], '--', color = 'Gray', linewidth = 1.5)
ax.plot([0,target_grid_polar[5,0]], [0,xlim], '--', color = 'Gray', linewidth = 1.5)

for i in range(len(radii1)):
    r = np.arange(-1, 1, 0.1)
    theta = np.zeros((r.shape)) + radii1[i]
    ax.plot(r,theta,'--', color = 'LightGray', linewidth = 2)

ax.bar(theta1, radii1, width=width1, bottom=0.0, color = colors1, alpha = 0.9)
ax.bar(theta2, radii2, width=width2, bottom=0.0, color = colors2, alpha=0.9)

ax.set_thetamin(-30)
ax.set_thetamax(30)
ax.set_rlim(0, xlim)

plt.yticks(ticks = [1, 2, 3, 4, 5, 6, 7], labels = ["22", "24", "26", "28", "30", "32", "36"])
plt.xticks(ticks = [-phi_unit, 0, phi_unit], labels = [-1, 0, 1])
ax.set_xlabel("Time (hrs APF)")
ax.yaxis.set_label_position("right")
plt.ylabel("Relative Angle (a.u.)", labelpad=35)

ax.grid(False)

if(is_save):
    fig_name = f'Figure3C.{fig_format}'
    fig_save_path = os.path.join(paths.output_prefix, fig_name)
    plt.savefig(fig_save_path, dpi=fig_res, bbox_inches='tight', format = fig_format, facecolor='w')
plt.show()

In [ ]:
# ====== stats ====== #
x_cat = 'time_id'
y_cat = 'angle_gc_mrr'
hue = 'type_plot'
df_plot = sum_df_ctrl_final[[x_cat, y_cat, hue]].dropna(how='any')
df_group = df_plot.groupby(x_cat)
for gp in df_group.groups.keys():
    print(f'{gp} hrs')
    my_help.stats_fig3b(df_group.get_group(gp), hue, y_cat, [['R3', 'R4']], 'Mann-Whitney')

### Figure 4
Extension speed is instrumental for asymmetrical targeting.

In [ ]:
pert_cat = 'Fz'
time_id = 24

if(pert_cat == 'Nic'):
    pert_type = 'R4/R4'
    pert_rtype = 4
    plot_df = sum_df_nic_final.groupby('time_id').get_group(time_id)
elif(pert_cat == 'Fz'):
    pert_type = 'R3/R3'
    pert_rtype = 3
    plot_df = sum_df_fz_final.groupby('time_id').get_group(time_id)

pert_info = time_id, pert_cat, pert_type, pert_rtype
my_help.generate_summary_polar_figure(plot_df, pert_info,  
                                      is_save = is_save, fig_format = fig_format, fig_res = fig_res)

In [ ]:
pert_cat = 'Fz'
time_id = 28

if(pert_cat == 'Nic'):
    pert_type = 'R4/R4'
    pert_rtype = 4
    plot_df = sum_df_nic_final.groupby('time_id').get_group(time_id)
elif(pert_cat == 'Fz'):
    pert_type = 'R3/R3'
    pert_rtype = 3
    plot_df = sum_df_fz_final.groupby('time_id').get_group(time_id)

pert_info = time_id, pert_cat, pert_type, pert_rtype
my_help.generate_summary_polar_figure(plot_df, pert_info, 
                                      is_save = is_save, fig_format = fig_format, fig_res = fig_res)

In [ ]:
pert_cat = 'Nic'
time_id = 24

if(pert_cat == 'Nic'):
    pert_type = 'R4/R4'
    pert_rtype = 4
    plot_df = sum_df_nic_final.groupby('time_id').get_group(time_id)
elif(pert_cat == 'Fz'):
    pert_type = 'R3/R3'
    pert_rtype = 3
    plot_df = sum_df_fz_final.groupby('time_id').get_group(time_id)

pert_info = time_id, pert_cat, pert_type, pert_rtype
my_help.generate_summary_polar_figure(plot_df, pert_info, 
                                     is_save = is_save, fig_format = fig_format, fig_res = fig_res)

In [ ]:
pert_cat = 'Nic'
time_id = 28

if(pert_cat == 'Nic'):
    pert_type = 'R4/R4'
    pert_rtype = 4
    plot_df = sum_df_nic_final.groupby('time_id').get_group(time_id)
elif(pert_cat == 'Fz'):
    pert_type = 'R3/R3'
    pert_rtype = 3
    plot_df = sum_df_fz_final.groupby('time_id').get_group(time_id)

pert_info = time_id, pert_cat, pert_type, pert_rtype
my_help.generate_summary_polar_figure(plot_df, pert_info, 
                                      is_save = is_save, fig_format = fig_format, fig_res = fig_res)

In [ ]:
# ====== stats: Fz, 24 hrs ====== #
pert_cat = 'Fz'
time_id = 24
x_cat = "type_plot"

if(pert_cat == 'Nic'):
    plot_df = sum_df_nic_final.groupby('time_id').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R4R4') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
elif(pert_cat == 'Fz'):
    plot_df = sum_df_fz_final.groupby('time_id').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R3R3') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]

my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'count')
my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'length')
stat = my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'angle')

In [ ]:
# ====== stats: Fz, 28 hrs ====== #
pert_cat = 'Fz'
time_id = 28
x_cat = "type_plot"

if(pert_cat == 'Nic'):
    plot_df = sum_df_nic_final.groupby('time_id').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R4R4') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
elif(pert_cat == 'Fz'):
    plot_df = sum_df_fz_final.groupby('time_id').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R3R3') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
    

my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'count')
my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'length')
my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'angle')

In [ ]:
# ====== stats: Nic, 24 hrs ====== #
pert_cat = 'Nic'
time_id = 24
x_cat = "type_plot"

if(pert_cat == 'Nic'):
    plot_df = sum_df_nic_final.groupby('time_id').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R4R4') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
elif(pert_cat == 'Fz'):
    plot_df = sum_df_fz_final.groupby('time_id').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R3R3') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
    

my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'count')
my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'length')
my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'angle')

In [ ]:
# ====== stats: Nic, 28 hrs ====== #
pert_cat = 'Nic'
time_id = 28
x_cat = "type_plot"

if(pert_cat == 'Nic'):
    plot_df = sum_df_nic_final.groupby('time_id').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R4R4') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
elif(pert_cat == 'Fz'):
    plot_df = sum_df_fz_final.groupby('time_id').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R3R3') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
    

my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'count')
my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'length')
my_help.stats_fig4(df_current, x_cat, pert_cat, time_id, 'angle')

## Supplemental Figures

### S2 Fig
Extension phenotype of wild-type flies over time.

In [ ]:
## 22 hrs
my_help.generate_density_plot_figure(output_data_ctrl_eg[('FigS2', 22, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.2, [0, 0.1, 0.2]], rfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]],
                                     fig_name = 'S2Fig_22hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 24 hrs
my_help.generate_density_plot_figure(output_data_ctrl_eg[('FigS2', 24, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]],
                                     fig_name = 'S2Fig_24hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 26 hrs
my_help.generate_density_plot_figure(output_data_ctrl_eg[('FigS2', 26, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.25, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]],
                                     fig_name = 'S2Fig_26hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 28 hrs
my_help.generate_density_plot_figure(output_data_ctrl_eg[('FigS2', 28, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.15, [0, 0.1]], rfp_cmap = [0.2, [0, 0.1, 0.2]],
                                     fig_name = 'S2Fig_28hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 30 hrs
my_help.generate_density_plot_figure(output_data_ctrl_eg[('FigS2', 30, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.2, [0, 0.1, 0.2]],
                                     fig_name = 'S2Fig_30hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 32 hrs
my_help.generate_density_plot_figure(output_data_ctrl_eg[('FigS2', 32, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.2, [0, 0.1, 0.2]], rfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]],
                                     fig_name = 'S2Fig_32hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 36 hrs
my_help.generate_density_plot_figure(output_data_ctrl_eg[('FigS2', 36, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.4, [0, 0.2, 0.4]], rfp_cmap = [0.15, [0, 0.1]],
                                     fig_name = 'S2Fig_36hrs', fig_format = fig_format, fig_res = fig_res)

### S3 Fig
Variation of the standardized coordinate system across samples.

In [ ]:
### general set-up
time_id = 26
annots_df_time_gp = annots_df_ctrl_final.groupby(['time_id'])
annots_df_time = annots_df_time_gp.get_group(time_id)
annots_df_smp_gp = annots_df_time.groupby('sample_no')

#### S3B: Raw measurements

In [ ]:
heel_coords, target_coords= my_help.plot_sample_variation_raw_measurements(annots_df_smp_gp, 
                                       is_save = False, fig_format = fig_format, fig_res = fig_res)

#### S3C: Polar plots

In [ ]:
coords, sum_coords = my_help.plot_sample_variation_polar(annots_df_smp_gp, 
                            is_save = False, fig_format = fig_format, fig_res = fig_res)

#### S3D: Overlay

In [ ]:
my_help.plot_sample_variation_polar_centroids(sum_coords, 
                                             is_save = False, fig_format = fig_format, fig_res = fig_res)

### Fig S4
Variation of the standardized coordinate system across time points.

In [ ]:
heel_centroid_sum, target_centroid_sum, coords_sum = my_help.plot_time_variation_raw_centroids(annots_df_ctrl_final, 
                                                                                       is_save = False, fig_format = fig_format, fig_res = fig_res)

#### S4B: Raw centroids overlay 

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8,4))

### params
index_to_target_id = settings.matching_info.index_to_target_id

### Plot reference
ax.plot([-5, 20], [0,0], '--', color = 'gray')

### Plot R and L
for i in range(len(annots_df_time_gp)):
    ### increase size over time
    dotsize = 2 + 2*i
    circlesie = 15 + 2*i

    for j in range(6):
        ### R cells
        ax.plot(heel_centroid_sum[j, 0, i], heel_centroid_sum[j, 1, i], 
                'o', color = color_code[j+1], alpha = 0.5, markersize = dotsize)
        ax.plot(target_centroid_sum[j, 0, i], target_centroid_sum[j, 1, i], 
                'o', mec = color_code[index_to_target_id[j]],
                markersize = circlesie, mew = 1, mfc = 'none')
### Plot Center
ax.plot(0, 0, 'o', color = 'k', markersize = 10)
ax.text(0.2, -0.8, "C")
    
### Axis
ax.set_xlim([-5, 15])
ax.set_ylim([5, -5])
ax.set_xlim([-4, 16])
ax.set_ylim([-5, 5])
ax.set_aspect(aspect=1)
ax.set_xlabel('X (um)')
ax.set_ylabel('y (um)')
ax.set_title('Raw Data Overlay')

### Saving
fig_name = f'S4B_Fig.{fig_format}'
fig_save_path = os.path.join(paths.output_prefix, fig_name)
if(is_save):
    plt.savefig(fig_save_path, dpi = fig_res, bbox_inches = 'tight', format = fig_format)

plt.show()

#### S4D: Coordinate centroids overlay

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_axes([0.1, 0.1, 0.75, 0.79], polar=True)

### params
plot_color = {
    'R3':color_code[3],
    'R4':color_code[4],
    'T4':color_code[4],
    'T3':color_code[3],
    'T7':color_code[7],
}


### Get data
phi_unit = my_help.get_angle_unit_theory('phi_unit')

sum_coords = {}
for t in plot_color.keys():
    sum_coords[t] = np.zeros((2, len(annots_df_time_gp.groups.keys())))

for i_time, time_id in enumerate(annots_df_time_gp.groups.keys()):
    for t in plot_color.keys():
        sum_coords[t][:, i_time] = np.mean(coords_sum[time_id][t], axis = 1)

### Plot references
ax.plot([0,0], [0,2.5], '--', color = "0.8", linewidth = 1)
ax.plot([0,sum_coords["T3"][0,0]], [0,2.5], '--', color = "0.8", linewidth = 1)
ax.plot([0,sum_coords["T7"][0,0]], [0,2.5], '--', color = "0.8", linewidth = 1)


### Plot centroids
for i_smp in range(len(annots_df_time_gp.groups.keys())):
    ### increase size over time
    dotsize = 2 + 2*i_smp
    circlesie = 15 + 2*i_smp

    for t in ['R3', 'R4']:
        ax.plot(sum_coords[t][0, i_smp], sum_coords[t][1, i_smp], 
             'o', color = plot_color[t], alpha = 0.5, markersize = dotsize)
    
    for t in ['T3', 'T4', 'T7']:
        ax.plot(sum_coords[t][0, i_smp], sum_coords[t][1, i_smp], 
             'o', mec = plot_color[t], markersize = circlesie, mew = 1.0, alpha = 0.8, mfc = 'none')

### Plot center
ax.plot(0, 0, 'o', color = 'k', markersize = 15)
ax.text(-0.1, -0.1, "C")

### Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 8, box.height])

### Axis
ax.set_thetamin(-30)
ax.set_thetamax(30)
ax.set_rlim(0, 2)
ax.set_yticks([0, 0.5, 1.0, 1.5, 2.0])
ax.set_yticklabels(["C", "0.5", "1.0", "1.5", "2.0"])
ax.set_xticks([-phi_unit, 0, phi_unit])
ax.set_xticklabels([1, 0, -1])
ax.grid(axis = 'y', linestyle = '--', which = 'major', linewidth = 0.5)
ax.grid(axis = 'x', linestyle = '--', which = 'major', linewidth = 0.5)
ax.set_xlabel("\nRelative Length (a.u.)")
ax.yaxis.set_label_position("right")
plt.ylabel("Relative Angle (a.u.)", labelpad=35)

### Saving
fig_name = f'S4D_Fig.{fig_format}'
fig_save_path = os.path.join(paths.output_prefix, fig_name)
if(is_save):
    plt.savefig(fig_save_path, dpi=300, bbox_inches='tight', format = fig_format)

plt.show()

### Figure S5
Extension phenotype of sev>Fz and sev>Nic flies over time.

In [ ]:
## sv>Fz 24 hrs, ctrl.
my_help.generate_density_plot_figure(output_data_fz_eg[('FigS5', 24, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.35, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.1, [0, 0.05, 0.1]],
                                     fig_name = 'S5Fig_Fz_24hrs_ctrl', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## sev>Fz 24 hrs, pert.
my_help.generate_density_plot_figure(output_data_fz_eg[('FigS5', 24, 'pert')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.2, [0, 0.1, 0.2]], rfp_cmap = [0.15, [0, 0.1]],
                                     fig_name = 'S5Fig_Fz_24hrs_pert', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## sev>Nic 24 hrs, ctrl.
my_help.generate_density_plot_figure(output_data_nic_eg[('FigS5', 24, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.4, [0, 0.2, 0.4]],
                                     fig_name = 'S5Fig_Nic_24hrs_ctrl', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## sev>Nic 24 hrs, pert.
my_help.generate_density_plot_figure(output_data_nic_eg[('FigS5', 24, 'pert')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.25, [0, 0.1, 0.2]],
                                     fig_name = 'S5Fig_Nic_24hrs_pert', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## sev>Fz 28 hrs, ctrl.
my_help.generate_density_plot_figure(output_data_fz_eg[('FigS5', 28, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.25, [0, 0.1, 0.2]], rfp_cmap = [0.2, [0, 0.1, 0.2]],
                                     fig_name = 'S5Fig_Fz_28hrs_ctrl', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## sev>Fz 28 hrs, pert.
my_help.generate_density_plot_figure(output_data_fz_eg[('FigS5', 28, 'pert')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.2, [0, 0.1, 0.2]], rfp_cmap = [0.15, [0, 0.1]],
                                     fig_name = 'S5Fig_Fz_28hrs_pert', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## sev>Nic 28 hrs, ctrl.
my_help.generate_density_plot_figure(output_data_nic_eg[('FigS5', 28, 'ctrl')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.15, [0, 0.1]], rfp_cmap = [0.25, [0, 0.1, 0.2]],
                                     fig_name = 'S5Fig_Nic_28hrs_ctrl', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## sev>Nic 28 hrs, pert.
my_help.generate_density_plot_figure(output_data_nic_eg[('FigS5', 28, 'pert')], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.25, [0, 0.1, 0.2]], rfp_cmap = [0.35, [0, 0.1, 0.2, 0.3]],
                                     fig_name = 'S5Fig_Nic_28hrs_pert', fig_format = fig_format, fig_res = fig_res)

### Figure S6
Repulsion model for determining growth cone extension angle.

In [ ]:
my_help.mutual_repulsion_regression_plot(sum_df_ctrl_ml, 
                                         is_save = is_save, fig_name = 'S6_Fig', 
                                         fig_format = fig_format, fig_res = fig_res)